In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import glob
import json

import pandas as pd

In [ ]:
import os

big_data = pd.DataFrame()
## find current folder root
root = os.getcwd()

all_files = glob.iglob('downloaded_configs/passivbot_v5.8.0/configs' + '/**/result.txt', recursive=True)


files = []
for file in all_files:
    if '\\old\\' not in file:
        files.append(file)

i = 0
for filename in files:
    #print(f'parsing : {filename}')
    json_file = os.path.abspath(filename.replace('result.txt', '../config.json'))
    with open(json_file) as f:
        live_config = json.load(f)
        config_type = live_config['config_name']

    with open(filename, 'r') as file:
        data = file.read().split('\n')
        d = {'file': filename}
        current_state = 'long'
        for l in data:
            l = l.split('|')
            if len(l) == 4:
                key = l[1].strip()
                value = l[2].strip()

                if key == 'Symbol':
                    d['s'] = value

                if key == 'Long':
                    current_state = 'long'
                    d['long'] = bool(value) or False
                if key == 'Short':
                    current_state = 'short'
                    d['short'] = bool(value) or False

                if key == 'Total gain' and current_state == 'long':
                    d['tgl_%'] = float(value.replace('%', '').strip())
                if key == 'Average daily gain' and current_state == 'long':
                    d['adgl_%'] = float(value.replace('%', '').strip())
                if key == 'Closest bankruptcy' and current_state == 'long':
                    d['bnkrl_%'] = float(value.replace('%', '').strip())
                if key == 'Lowest equity/balance ratio' and current_state == 'long':
                    d['ebrl'] = float(value.replace('%', '').strip())
                if key == 'Max hours stuck' and current_state == 'long':
                    d['maxsl'] = float(value.replace('%', '').strip()) / 24
                if key == 'Price action distance max' and current_state == 'long':
                    d['pamaxl'] = float(value.replace('%', '').strip())
                if key == 'Price action distance std' and current_state == 'long':
                    d['pastdl'] = float(value.strip())

                if key == 'Total gain' and current_state == 'short':
                    d['tgs_%'] = float(value.replace('%', '').strip())
                if key == 'Average daily gain' and current_state == 'short':
                    d['adgs_%'] = float(value.replace('%', '').strip())
                if key == 'Closest bankruptcy' and current_state == 'short':
                    d['bnkrs_%'] = float(value.replace('%', '').strip())
                if key == 'Lowest equity/balance ratio' and current_state == 'short':
                    d['ebrs'] = float(value.replace('%', '').strip())
                if key == 'Max hours stuck' and current_state == 'short':
                    d['maxss'] = float(value.replace('%', '').strip()) / 24
                if key == 'Price action distance max' and current_state == 'short':
                    d['pamaxs'] = float(value.replace('%', '').strip())
                if key == 'Price action distance std' and current_state == 'short':
                    d['pastds'] = float(value.strip())

        big_data = big_data.append(d, ignore_index=True)
big_data

In [ ]:
allowed_symbols = [
    "ETHUSDT", "BCHUSDT", "XRPUSDT", "EOSUSDT", "LTCUSDT", "TRXUSDT", "ETCUSDT", "LINKUSDT", "XLMUSDT", "ADAUSDT",
    "XMRUSDT", "DASHUSDT", "ZECUSDT", "XTZUSDT", "BNBUSDT", "ATOMUSDT", "NEOUSDT", "IOSTUSDT", "THETAUSDT", "ALGOUSDT",
    "ZILUSDT", "KNCUSDT", "COMPUSDT", "OMGUSDT", "DOGEUSDT", "KAVAUSDT", "BANDUSDT", "RLCUSDT", "WAVESUSDT", "SNXUSDT",
    "DOTUSDT", "CRVUSDT", "TRBUSDT", "RUNEUSDT", "SUSHIUSDT", "SRMUSDT", "EGLDUSDT", "FTMUSDT", "HNTUSDT", "RENUSDT",
    "KSMUSDT", "NEARUSDT", "FILUSDT", "RSRUSDT", "MATICUSDT", "OCEANUSDT", "CVCUSDT", "GRTUSDT", "1INCHUSDT", "CHZUSDT",
    "SANDUSDT", "BTSUSDT", "LITUSDT", "UNFIUSDT", "REEFUSDT", "MANAUSDT", "HBARUSDT", "ONEUSDT", "LINAUSDT", "STMXUSDT",
    "DENTUSDT", "HOTUSDT", "MTLUSDT", "SCUSDT", "1000SHIBUSDT"
]
#allowed_symbols = [s + "USDT" for s in allowed_symbols]
big_data['adgt_%'] = big_data['adgl_%'] + big_data['adgs_%']
df = big_data[(big_data.s.isin(allowed_symbols))
    #& (big_data['long'] == True)
    #& (big_data['adgl_%'] > 0.04)
    #& (big_data['short'] == True)
    #& (big_data['adgt_%'] > 0.1)
    #& (big_data['bnkrl_%'] > 95.0)
    #& (big_data['bnkrs_%'] > 95.0)
    #& (big_data['ebrs'] > .85)
    #& (big_data['ebrl'] > .8)
    #& (big_data['v'] == '5.5')

    # & (big_data['tg_%'] > 10)
    # & (big_data['ebr'] > .85)
]
#df['total_adg'] = big_data['adgl_%'] + big_data['adgs_%']
df.to_csv('5.8.csv')
df

In [5]:
df.drop_duplicates(
    subset=['s',  'tgl_%', 'tgs_%', 'adgl_%', 'adgs_%', 'adgt_%', 'bnkrl_%', 'bnkrs_%', 'ebrl', 'ebrs', 'long',
            'short'], keep='first',
    inplace=True)
dx = df[['file',  's', 'adgt_%', 'adgl_%', 'adgs_%', 'bnkrl_%', 'bnkrs_%', 'ebrl', 'ebrs', 'maxsl', 'pamaxl','pastdl',
         'maxss', 'pamaxs','pastds' ]]
dx

C:\Users\skhan\AppData\Local\Temp\ipykernel_4832\994885060.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop_duplicates(


,file,s,adgt_%,adgl_%,adgs_%,bnkrl_%,bnkrs_%,ebrl,ebrs,maxsl,pamaxl,pastdl,maxss,pamaxs,pastds
0,downloaded_configs/passivbot_v5.8.0/configs\ha...,ATOMUSDT,NaN,0.129,NaN,99.74,NaN,0.3945,NaN,6.218750,1.535200,0.030567,NaN,NaN,NaN
1,downloaded_configs/passivbot_v5.8.0/configs\ha...,CHZUSDT,NaN,0.266,NaN,99.81,NaN,0.3294,NaN,37.304167,2.036520,0.162400,NaN,NaN,NaN
2,downloaded_configs/passivbot_v5.8.0/configs\ha...,DOGEUSDT,NaN,0.424,NaN,99.99,NaN,0.5936,NaN,4.569458,0.684707,0.010823,NaN,NaN,NaN
3,downloaded_configs/passivbot_v5.8.0/configs\ha...,DOTUSDT,NaN,0.300,NaN,99.77,NaN,0.5217,NaN,11.056958,0.915960,0.017572,NaN,NaN,NaN
5,downloaded_configs/passivbot_v5.8.0/configs\ha...,EOSUSDT,NaN,0.173,NaN,99.98,NaN,0.6917,NaN,14.597917,0.445603,0.013177,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
538,downloaded_configs/passivbot_v5.8.0/configs\pa...,HBARUSDT,NaN,0.186,NaN,99.83,NaN,0.6671,NaN,9.752083,0.498154,0.011753,NaN,NaN,NaN
539,downloaded_configs/passivbot_v5.8.0/configs\pa...,MATICUSDT,NaN,0.261,NaN,99.85,NaN,0.7691,NaN,8.481958,0.299799,0.012178,NaN,NaN,NaN
540,downloaded_configs/passivbot_v5.8.0/configs\pa...,SUSHIUSDT,NaN,0.228,NaN,99.75,NaN,0.5204,NaN,2.986804,0.919427,0.010983,NaN,NaN,NaN
541,downloaded_configs/passivbot_v5.8.0/configs\pa...,XRPUSDT,NaN,0.223,NaN,99.86,NaN,0.7536,NaN,8.080542,0.326524,0.009385,NaN,NaN,NaN


In [7]:
dl_proposed = dx[
    #(dx['lm'] == 'n') &
    (dx['bnkrl_%'] == 100) &
    (dx['bnkrs_%'] == 100) &
    (dx['ebrl'] > 0.9) &  #&
    (dx['ebrs'] > 0.9) &  #&
    # (dx['adgl_%'] > 0.04) 
    (dx['pastdl'] < 0.03) &
    (dx['pastds'] < 0.03) &  #
    (dx['maxsl'] < 10) &
    (dx['maxss'] < 10) &
    (dx['adgt_%'].notnull())
    ]
#dl_proposed = dl_proposed[['file','s','adgl_%', 'bnkrl_%', 'ebrl',  'maxsl', 'pamaxl','pastdl' ]]
dl_proposed

,file,s,adgt_%,adgl_%,adgs_%,bnkrl_%,bnkrs_%,ebrl,ebrs,maxsl,pamaxl,pastdl,maxss,pamaxs,pastds
35,downloaded_configs/passivbot_v5.8.0/configs\ha...,HBARUSDT,0.07030,0.04080,0.02950,100.0,100.0,0.9468,0.9567,5.604875,0.292471,0.020113,6.254875,0.264259,0.019793
75,downloaded_configs/passivbot_v5.8.0/configs\ha...,LINKUSDT,0.06760,0.05180,0.01580,100.0,100.0,0.9329,0.9762,6.122917,0.365803,0.017020,6.031958,0.164406,0.014173
77,downloaded_configs/passivbot_v5.8.0/configs\ha...,MATICUSDT,0.14260,0.12000,0.02260,100.0,100.0,0.9043,0.9536,3.083333,0.618375,0.019128,5.791667,0.276874,0.015581
85,downloaded_configs/passivbot_v5.8.0/configs\ha...,MATICUSDT,0.13820,0.11500,0.02320,100.0,100.0,0.9030,0.9537,4.554167,0.631445,0.012852,5.102083,0.276613,0.017294
87,downloaded_configs/passivbot_v5.8.0/configs\ha...,ATOMUSDT,0.07170,0.05500,0.01670,100.0,100.0,0.9120,0.9530,9.906250,0.542391,0.019779,8.981250,0.280822,0.019068
93,downloaded_configs/passivbot_v5.8.0/configs\ha...,HBARUSDT,0.10180,0.06520,0.03660,100.0,100.0,0.9004,0.9544,3.570137,0.660923,0.008968,4.074304,0.274874,0.011451
96,downloaded_configs/passivbot_v5.8.0/configs\ha...,MATICUSDT,0.13930,0.11600,0.02330,100.0,100.0,0.9367,0.9656,4.178458,0.338400,0.010511,4.292375,0.222335,0.014170
100,downloaded_configs/passivbot_v5.8.0/configs\ha...,XRPUSDT,0.06410,0.04640,0.01770,100.0,100.0,0.9414,0.9460,9.789583,0.305723,0.008553,5.250708,0.309627,0.010707
132,downloaded_configs/passivbot_v5.8.0/configs\ot...,MATICUSDT,0.02318,0.01480,0.00838,100.0,100.0,0.9807,0.9750,4.659708,0.281597,0.016793,5.259042,0.199938,0.019015
133,downloaded_configs/passivbot_v5.8.0/configs\ot...,ATOMUSDT,0.02500,0.01280,0.01220,100.0,100.0,0.9517,0.9724,4.167375,0.939702,0.023376,4.456250,0.217075,0.019573
